In [1]:
!pip install openai


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [67]:
import openai
import getpass
import json
import re
import pandas as pd
import string

In [8]:
openai.api_key = getpass.getpass(prompt='Please enter your OpenAI API key.')
openai.organization = getpass.getpass(prompt='Please enter your OpenAI Organization key.')

## New Spanish alignment


- [ ] Fix the output parsing part of the notebook. Perhaps make a function called `get_alignments_from_prompt_output()` instead of just splitting on highly specific string values
- [ ] (TODO LATER) Programmatically load a Spanish translation from eBible corpus


Some ideas for improving the initial prompt.

- [x] Add language typology information about Spanish
- [x] Add information about translation style of specific Spanish translation we are using (more literal vs. more dynamic)
- [x] Add more examples?
- [x] Replace current examples with examples from the language (e.g., Spanish)
- [x] Move the rationale of each alignment before the alignment itself
- [x] Experiment with tweaking the system prompt to see if it helps
- [x] SIMPLIFY the prompt down to just the most relevant content (do we need a rationale? Does it work without? We can probably drop out relevant grammatical patterns)

In [5]:
# Read the file
with open('spapddpt.json', encoding='utf-8') as file:
    data = json.load(file)

# Create a new list for units with vref MRK 1
data_list = []

# Iterate through the units
for unit in data:
    if 'MRK 1:' in unit['vref']:
        data_list.append(unit)

In [7]:
data_list[:5]

[{'vref': 'MRK 1:1',
  'bsb': {'vref': 'MRK 1:1',
   'content': 'This is the beginning of the gospel of Jesus Christ, the Son of God.'},
  'macula': {'vref': 'MRK 1:1',
   'content': 'Ἀρχὴ  τοῦ  εὐαγγελίου  Ἰησοῦ  Χριστοῦ  Υἱοῦ  Θεοῦ.'},
  'target': {'vref': 'MRK 1:1',
   'content': 'Principio de las Buenas Noticias de Jesucristo.'}},
 {'vref': 'MRK 1:2',
  'bsb': {'vref': 'MRK 1:2',
   'content': 'As it is written in Isaiah the prophet: “Behold, I will send My messenger ahead of You, who will prepare Your way.”'},
  'macula': {'vref': 'MRK 1:2',
   'content': 'Καθὼς  γέγραπται  ἐν  τῷ  Ἠσαΐᾳ  τῷ  προφήτῃ  Ἰδοὺ  ἀποστέλλω  τὸν  ἄγγελόν  μου  πρὸ  προσώπου  σου, ὃς  κατασκευάσει  τὴν  ὁδόν  σου·'},
  'target': {'vref': 'MRK 1:2',
   'content': 'Como está escrito en el profeta Isaías: Ciertamente envío mi mensajero delante de Ti, quien preparará tu camino.'}},
 {'vref': 'MRK 1:3',
  'bsb': {'vref': 'MRK 1:3',
   'content': '“A voice of one calling in the wilderness, ‘Prepare the way for 

In [11]:
def generate_prompt(data):
    eng = data['bsb']['content']
    grk = data['macula']['content']
    spa = data['target']['content']
    
    prompt = f'''
    Here are some general facts to note about Spanish:
    Spanish is a fusional language, ensure correct affix attachment; follow SVO order; mark verbs for tense, aspect, mood.
    For translating from Greek: replace Greek's three-gender system with Spanish's two-gender system, ensuring agreement; shift to SVO order; adapt Greek Voice/Aspect/Mood markings to Spanish system.

    Translation style:
    The Spanish translation is a balanced approach between a literal translation and a more dynamic translation. This means that there may be slight differences in phrase order and other adaptations to produce a more natural translation.

    Here is a sentence:
    English: And He said to them What things; - And they said to Him The things concerning Jesus of Nazareth, who was a man a prophet mighty in deed and word before - God and all the people,
    Greek: καὶ εἶπεν αὐτοῖς Ποῖα;οἱ δὲ εἶπαν αὐτῷ Τὰ περὶ Ἰησοῦ τοῦ Ναζαρηνοῦ,ὃς ἐγένετο ἀνὴρ προφήτης δυνατὸς ἐν ἔργῳ καὶ λόγῳ ἐναντίον τοῦ Θεοῦ καὶ παντὸς τοῦ λαοῦ,
    Spanish: —¿Qué es lo que ha pasado? —preguntó. Ellos respondieron: —Lo de Jesús de Nazaret. Era un profeta poderoso en obras y en palabras delante de Dios y de todo el pueblo.

    Here is a phonological, semantic, orthographic alignment of that sentence:
    ```
    [
        {{
            "Spanish phrase": "—preguntó.",
            "Target phrase": "And He said to them",
            "Greek phrase": "καὶ εἶπεν αὐτοῖς"
        }},
        {{
            "Spanish phrase": "¿Qué es lo que ha pasado?",
            "Target phrase": "What things;",
            "Greek phrase": "Ποῖα;"
        }},
        {{
            "Spanish phrase": "Ellos respondieron",
            "Target phrase": "And they said to Him",
            "Greek phrase": "οἱ δὲ εἶπαν αὐτοῖς"
        }},
        {{
            "Spanish phrase": "—Lo de Jesús de Nazaret.",
            "Target phrase": "The things concerning Jesus of Nazareth,",
            "Greek phrase": "Τὰ περὶ Ἰησοῦ τοῦ Ναζαρηνοῦ"
        }},
        {{
            "Spanish phrase": "Era un profeta poderoso en obras y en palabras",
            "Target phrase": "who was a man a prophet mighty in deed and word",
            "Greek phrase": "ὃς ἐγένετο ἀνὴρ προφήτης δυνατὸς ἐν ἔργῳ καὶ λόγῳ"
        }},
        {{
            "Spanish phrase": "delante de Dios y de todo el pueblo.",
            "Target phrase": "before - God and all the people",
            "Greek phrase": "ἐναντίον τοῦ Θεοῦ καὶ παντὸς τοῦ λαοῦ"
        }}
    ]

    ```

    Please also align the following sentence. Avoid including multiple phrases in a single alignment unit. You may need to break phrases  on commas or other major punctuation, including enclosing quotation marks. But you may also need to break a phrase along conjunctions or other words that typically mark the start of a new phrase:

    English: {eng}
    Greek: {grk}
    Spanish: {spa}
    '''
    
    return prompt

In [68]:
def strip_punctuation(input_string):
    # Create a translation table to remove punctuation
    translation_table = str.maketrans("", "", string.punctuation)

    # Use translate to remove punctuation
    stripped_string = input_string.translate(translation_table)

    return stripped_string

In [12]:
alignments = []

for data in data_list:
    prompt = generate_prompt(data)

    messages = [
        {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.3,
        n=1,
        presence_penalty=0.5,
        frequency_penalty=0.5,
    )

    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    print(generated_texts[0])
    alignments.append(generated_texts)

[
    {
        "Spanish phrase": "Principio de las Buenas Noticias",
        "Target phrase": "This is the beginning of the gospel",
        "Greek phrase": "Ἀρχὴ  τοῦ  εὐαγγελίου"
    },
    {
        "Spanish phrase": "de Jesucristo.",
        "Target phrase": "of Jesus Christ,",
        "Greek phrase": "Ἰησοῦ  Χριστοῦ"
    },
    {
        "Spanish phrase": "",
        "Target phrase": "the Son of God.",
        "Greek phrase": "Υἱοῦ  Θεοῦ."
    }
]
Here is the alignment of the given sentence:

```
[
    {
        "Spanish phrase": "Como está escrito en el profeta Isaías:",
        "Target phrase": "As it is written in Isaiah the prophet:",
        "Greek phrase": "Καθὼς γέγραπται ἐν τῷ Ἠσαΐᾳ τῷ προφήτῃ"
    },
    {
        "Spanish phrase": "Ciertamente envío mi mensajero delante de Ti,",
        "Target phrase": "“Behold, I will send My messenger ahead of You,",
        "Greek phrase": "Ἰδοὺ ἀποστέλλω τὸν ἄγγελόν μου πρὸ προσώπου σου,"
    },
    {
        "Spanish phrase": "qui

## Align individual chunks from output

In [26]:
def get_alignments_from_prompt_output(generated_texts):
  pattern = r'\[(.*?)\]'

  result = re.search(pattern, generated_texts, re.DOTALL).group(1)
  
  # json.dumps(data, indent=4, ensure_ascii=False)
  return result


In [28]:
aligned_data = []

for i in alignments:
    aligned_data.append(get_alignments_from_prompt_output(i[0]))

In [30]:
aligned_data[:5]

['\n    {\n        "Spanish phrase": "Principio de las Buenas Noticias",\n        "Target phrase": "This is the beginning of the gospel",\n        "Greek phrase": "Ἀρχὴ  τοῦ  εὐαγγελίου"\n    },\n    {\n        "Spanish phrase": "de Jesucristo.",\n        "Target phrase": "of Jesus Christ,",\n        "Greek phrase": "Ἰησοῦ  Χριστοῦ"\n    },\n    {\n        "Spanish phrase": "",\n        "Target phrase": "the Son of God.",\n        "Greek phrase": "Υἱοῦ  Θεοῦ."\n    }\n',
 '\n    {\n        "Spanish phrase": "Como está escrito en el profeta Isaías:",\n        "Target phrase": "As it is written in Isaiah the prophet:",\n        "Greek phrase": "Καθὼς γέγραπται ἐν τῷ Ἠσαΐᾳ τῷ προφήτῃ"\n    },\n    {\n        "Spanish phrase": "Ciertamente envío mi mensajero delante de Ti,",\n        "Target phrase": "“Behold, I will send My messenger ahead of You,",\n        "Greek phrase": "Ἰδοὺ ἀποστέλλω τὸν ἄγγελόν μου πρὸ προσώπου σου,"\n    },\n    {\n        "Spanish phrase": "quien preparará tu cam

In [33]:
with open('posalign_stage_1.json', 'a', encoding='utf-8') as file:
    for i in aligned_data:
        file.write(i + '\n')

In [41]:
final_alignments = []

for alignment in aligned_data:
  tokenized_data = []
  
  pattern = r'\{([^}]+)\}'
  chunks = re.findall(pattern, alignment)
  
  for chunk in chunks:
    
    phrase = strip_punctuation(alignment)
    chunk = strip_punctuation(chunk)

    prompt = f'''Here is a phrase:
        {phrase}

        Please further align and break down the provided chunk into a JSON mapping of the fewest possible tokens (sometimes multiple tokens will align to one token; that's expected) in a format similar to the following:

        ```
        [
          {{
            "source": "Después de",
            "target": "After now",
            "macula": "Μετὰ δὲ"
          }},
          {{
            "source": "esto",
            "target": "these things",
            "macula": "ταῦτα"
          }},
          ...
        ]
        ```

        Do not include any other information or commentary. Only tell me what the alignment is.

        The chunk I want you to align is:
          ```
          {chunk}
          ```
        '''
      
    messages = [
          {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
          {"role": "user", "content": prompt},
      ]

    response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=messages,
          temperature=0.3,
          n=1,
          presence_penalty=0.5,
          frequency_penalty=0.5,
      )

    generated_texts_for_chunk = [
          choice.message["content"].strip() for choice in response["choices"]
      ]
    print(generated_texts_for_chunk[0])
    tokenized_data.append(generated_texts_for_chunk[0])
  final_alignments.append(tokenized_data)


        "Spanish phrase": "Principio de las Buenas Noticias",
        "Target phrase": "This is the beginning of the gospel",
        "Greek phrase": "Ἀρχὴ  τοῦ  εὐαγγελίου"
    
[
  {
    "source": "Principio",
    "target": "This is the beginning",
    "macula": "Ἀρχὴ"
  },
  {
    "source": "de las Buenas Noticias",
    "target": "of the gospel",
    "macula": "τοῦ εὐαγγελίου"
  }
]

        "Spanish phrase": "de Jesucristo.",
        "Target phrase": "of Jesus Christ,",
        "Greek phrase": "Ἰησοῦ  Χριστοῦ"
    
[
  {
    "source": "de",
    "target": "of",
    "macula": "Ἰησοῦ"
  },
  {
    "source": "Jesucristo",
    "target": "Jesus Christ",
    "macula": "Χριστοῦ"
  }
]

        "Spanish phrase": "",
        "Target phrase": "the Son of God.",
        "Greek phrase": "Υἱοῦ  Θεοῦ."
    
[
  {
    "source": "",
    "target": "the Son of God",
    "macula": "Υἱοῦ Θεοῦ"
  }
]

        "Spanish phrase": "Como está escrito en el profeta Isaías:",
        "Target phrase": "As it i

In [42]:
final_alignments

[['[\n  {\n    "source": "Principio",\n    "target": "This is the beginning",\n    "macula": "Ἀρχὴ"\n  },\n  {\n    "source": "de las Buenas Noticias",\n    "target": "of the gospel",\n    "macula": "τοῦ εὐαγγελίου"\n  }\n]',
  '[\n  {\n    "source": "de",\n    "target": "of",\n    "macula": "Ἰησοῦ"\n  },\n  {\n    "source": "Jesucristo",\n    "target": "Jesus Christ",\n    "macula": "Χριστοῦ"\n  }\n]',
  '[\n  {\n    "source": "",\n    "target": "the Son of God",\n    "macula": "Υἱοῦ Θεοῦ"\n  }\n]'],
 ['[\n  {\n    "source": "Como",\n    "target": "As",\n    "macula": "Καθὼς"\n  },\n  {\n    "source": "está escrito en el profeta Isaías",\n    "target": "it is written in Isaiah the prophet",\n    "macula": "γέγραπται ἐν τῷ Ἠσαΐᾳ τῷ προφήτῃ"\n  }\n]',
  '[\n  {\n    "source": "Ciertamente",\n    "target": "Behold,",\n    "macula": "Ἰδοὺ"\n  },\n  {\n    "source": "envío mi mensajero delante de Ti,",\n    "target": "I will send My messenger ahead of You,",\n    "macula": "ἀποστέλλω τὸν ἄ

In [71]:
result = []

for chunks in final_alignments:
    for chunk in chunks:
        chunk = str(chunk)
        try:
            source = re.search(r'"source": "(.*?)"', str(chunk)).group(1)
            target = re.search(r'"target": "(.*?)"', str(chunk)).group(1)
            macula = re.search(r'"macula": "(.*?)"', str(chunk)).group(1)
            
            if len(source) < 1:
                source = "No Matches Found"
            if len(target) < 1:
                target = "No Matches Found"
            if len(macula) < 1:
                macula = "No Matches Found"
            
            result.append((source, target, macula))
        except:
            pass

In [72]:
result

[('Principio', 'This is the beginning', 'Ἀρχὴ'),
 ('de', 'of', 'Ἰησοῦ'),
 ('No Matches Found', 'the Son of God', 'Υἱοῦ Θεοῦ'),
 ('Como', 'As', 'Καθὼς'),
 ('Ciertamente', 'Behold,', 'Ἰδοὺ'),
 ('quien', 'who', 'ὃς'),
 ('Voz', 'A voice', 'φωνὴ'),
 ('Preparen', "'Prepare", 'Ἑτοιμάσατε'),
 ('Enderecen', 'make', 'εὐθείας'),
 ('Juan', 'John', 'Ἰωάνης'),
 ('Bautizaba', 'preaching', 'κηρύσσων'),
 ('un', 'for', 'εἰς'),
 ('Los', 'People', 'καὶ'),
 ('Confesaban', 'Confessing', 'ἐξομολογούμενοι'),
 ('y', 'they', 'καὶ'),
 ('Juan', 'John', 'Ἰωάνης'),
 ('y', 'His', 'καὶ'),
 ('Proclamaba', 'And he began to proclaim', 'καὶ ἐκήρυσσεν'),
 ('Viene tras mí', 'After me will come', 'λέγων Ἔρχεται'),
 ('de Quien', ', the straps of whose', ', οὗ'),
 ('Yo', 'I', 'ἐγὼ'),
 ('pero', 'but', 'αὐτὸς'),
 ('Aconteció', 'In', 'Καὶ'),
 ('que', 'Jesus', 'Ἰησοῦς'),
 ('y', 'and', 'καὶ'),
 ('De inmediato', 'As soon as', 'καὶ εὐθὺς'),
 ('al salir del agua',
  'Jesus came up out of the water',
  'ἀναβαίνων ἐκ τοῦ ὕδατος'),
 ('v

In [74]:
results_table = pd.DataFrame(result, columns=['Spanish', 'English', 'Greek'])

In [75]:
results_table.head()

,Spanish,English,Greek
0,Principio,This is the beginning,Ἀρχὴ
1,de,of,Ἰησοῦ
2,No Matches Found,the Son of God,Υἱοῦ Θεοῦ
3,Como,As,Καθὼς
4,Ciertamente,"Behold,",Ἰδοὺ


In [66]:
results_table.to_html('posalign_resultsv1.html')